In [1]:
import requests
import zipfile
import io
import os

def download_and_zip_github_docs(github_repo_path, output_zip_filename="documentation.zip"):
    """Downloads files from a GitHub repository and creates a zip archive (recursive)."""

    try:
        parts = github_repo_path.split("/")
        if "tree" not in parts:
            raise ValueError("Invalid GitHub path. Must use the 'tree' URL format.")

        tree_index = parts.index("tree")
        owner = parts[tree_index - 2]
        repo = parts[tree_index - 1]
        branch = parts[tree_index + 1]
        base_path = "/".join(parts[tree_index + 2:])


        def download_recursive(path, zip_file):
            response = requests.get(f"https://api.github.com/repos/{owner}/{repo}/contents/{path}?ref={branch}")
            response.raise_for_status()
            contents = response.json()

            for item in contents:
                if item["type"] == "file":
                    file_url = item["download_url"]
                    file_content = requests.get(file_url).content
                    zip_path = os.path.relpath(item["path"], base_path) # Correct path in zip
                    zip_file.writestr(zip_path, file_content)
                elif item["type"] == "dir":
                    download_recursive(item["path"], zip_file) # Recursive call for subs


        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, "w") as zip_file:
            download_recursive(base_path, zip_file)

        zip_buffer.seek(0)

        with open(output_zip_filename, "wb") as f:
            f.write(zip_buffer.read())

        print(f"Zip file '{output_zip_filename}' created successfully.")

    except requests.exceptions.RequestException as e:
        print(f"Error accessing GitHub API or downloading files: {e}")
    except zipfile.BadZipFile as e:
        print(f"Error creating zip file: {e}")
    except ValueError as e:
      print(f"Error: {e}")


# run dis:
github_repo_path = "huggingface/smolagents/tree/main/docs/source/en"
download_and_zip_github_docs(github_repo_path)

Zip file 'documentation.zip' created successfully.
